In [4]:
import pandas as pd

mvps = pd.read_csv('mvps.csv')
teams = pd.read_csv('teams.csv')
players = pd.read_csv('players.csv')

In [8]:
mvps = mvps[['Player', 'Year', 'Pts Won', 'Pts Max', 'Share']] # only need these columns from mvp csv (rest are in players.csv)

In [10]:
del players['Unnamed: 0']
del players['Rk']

In [15]:
players['Player'] = players['Player'].str.replace('*', '', regex=False) # cleaning * from player names


In [26]:
# function to only keep total stats
def get_single_row(player_group):
  if player_group.shape[0] == 1: return player_group

  total_row = player_group[player_group['Tm'] == 'TOT']
  total_row["Tm"] = player_group.iloc[-1,:]['Tm']
  return total_row

players = players.groupby(['Player', 'Year']).apply(get_single_row)

In [29]:
players.index = players.index.droplevel() # run twice

In [32]:
# outer merge allows us to keep data of players and years not in mvps
comb = players.merge(mvps, how='outer', on=['Player', 'Year'])

In [40]:
# need to clean up NaN
comb[['Pts Won', 'Pts Max', 'Share']] = comb[['Pts Won', 'Pts Max', 'Share']].fillna(0)

In [41]:
comb

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
0,A.C. Green,PF,27,LAL,82,21,26.4,3.1,6.6,.476,...,0.9,0.7,0.3,1.2,1.4,9.1,1991,0.0,0.0,0.0
1,A.C. Green,PF,28,LAL,82,53,35.4,4.7,9.8,.476,...,1.4,1.1,0.4,1.4,1.7,13.6,1992,0.0,0.0,0.0
2,A.C. Green,PF,29,LAL,82,55,34.4,4.6,8.6,.537,...,1.4,1.1,0.5,1.4,1.8,12.8,1993,0.0,0.0,0.0
3,A.C. Green,PF,30,PHO,82,55,34.5,5.7,11.3,.502,...,1.7,0.9,0.5,1.2,1.7,14.7,1994,0.0,0.0,0.0
4,A.C. Green,SF,31,PHO,82,52,32.8,3.8,7.5,.504,...,1.5,0.7,0.4,1.4,1.8,11.2,1995,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14087,Željko Rebrača,C,29,DET,74,4,15.9,2.6,5.1,.505,...,0.5,0.4,1.0,1.1,2.6,6.9,2002,0.0,0.0,0.0
14088,Željko Rebrača,C,30,DET,30,12,16.3,2.7,4.8,.552,...,0.3,0.2,0.6,1.0,2.6,6.6,2003,0.0,0.0,0.0
14089,Željko Rebrača,C,31,ATL,24,2,11.4,1.4,3.2,.442,...,0.3,0.2,0.5,0.7,2.2,3.8,2004,0.0,0.0,0.0
14090,Željko Rebrača,C,32,LAC,58,2,16.0,2.3,4.0,.568,...,0.4,0.2,0.7,0.8,2.2,5.8,2005,0.0,0.0,0.0
